# 네이버 웹툰

웹툰| https://comic.naver.com/webtoon

- [import](#import)
- [selenium setting](#setting)
- [문제와 해결방법 idea](#문제와-해결방법-idea)
- [완결 웹툰](#완결된-웹툰)
    * [웹툰 information](#웹툰-information)
    * [에피소드 별 평점](#각-웹툰-에피소드-당-평점)
    * [웹툰 별, 각 에피소드 댓글](#웹툰-별,-각-에피소드-댓글)
        + [중간에 멈추는 경우](#중간에-멈출경우)

# import

In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains # 스크롤 다운위해서
from selenium.webdriver.common.keys import Keys
import pyperclip
import time

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.support.select import Select

In [2]:
import sys
print(sys.version) # 3.9.12

import selenium
print(selenium.__version__) # 4.2.0

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
4.2.0


# setting

In [3]:
# 경로 설정
path = 'C:\chromedriver_win32\chromedriver.exe'
url = 'https://comic.naver.com/webtoon'

# 드라이버 설정
def drive_path(x, path):
    global driver # 전역변수 선언하기
    if x == 'chrome':
        driver = webdriver.Chrome(path)
    else:
        print('driver 새로 설정하세요')

# url 설정
def url_move(url):
    if isinstance(url, str):
        driver.get(url)
    else:
        driver.get(str(url))

In [4]:
drive_path('chrome',path)
print('드라이버 설정 완료')
print('-----------------------')

url_move(url)
print('url로 창 열기 완료')
print('-----------------------')
time.sleep(3) # seconds 기다리기

driver.set_window_size(1536, 1024) # 창 사이즈 변경

드라이버 설정 완료
-----------------------
url로 창 열기 완료
-----------------------


---

# 문제와 해결방법 idea

- 기다리면 무료인 작품은 클릭하면 바로 로그인 창이 뜸 → 웹툰에 기다리면 무료 아이콘이 존재할 경우 클릭하지 않고, pass하기 + 성인 제한 걸린 것도 같이 pass 됨 + 신작이라는 아이콘도같이 pass된다는 문제
- 웹툰 전체 에피소드가 무료인 경우도 있지만 일부만 무료이고, 유료인 경우도 있음. 유료인 회차 클릭 시 로그인 창으로 이동 → 유료인 경우, 회차 옆에 아이콘 존재. 있는 경우와 아닌 경우로 나눠서 (우선 다 무료라는 전제로 코드 짠 후, 수정하는 방향으로)
- best 댓글이 없는 경우도 존재
- 전체 댓글 약 200개 수집 시, 댓글이 200개가 안되는 경우 존재 → 전체 댓글 수 추출하여 활용
- 삭제된 댓글 존재

---

# 완결된 웹툰

인기순으로 Top 50 웹툰

In [5]:
global webtoon_info, comment_data, score_data
webtoon_info = pd.DataFrame(columns=['seq','title','descript','genre','author','artist','origin','total','star']) # 웹툰 정보
score_data = pd.DataFrame(columns=['title','episode','score','date']) # 에피소드 평점
comment_data = pd.DataFrame(columns=['title','episode','best_comment','comment','comment_num']) # 댓글
# 정보 모으고 평균 평점과 각 에피소드 평점 비교 후 semtiment 추가 작업하여 넣기

In [6]:
# 완결 웹툰 클릭 후, 인기 순으로 정렬하기
def finish_popular():
    # 완결 웹툰 클릭하기
    driver.find_element_by_xpath('//*[@id="wrap"]/header/div[3]/nav/ul/li[11]/a').click()
    time.sleep(1) # seconds 기다리기
    # 인기순으로 정렬
    driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/button[1]').click()
    time.sleep(1) # seconds 기다리기

In [7]:
# 무료 작품인지 아닌지 확인한 후 기다리면 무료작품일 경우는 pass 시키기
def free(num):
    di = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/a/div')
    find_i = di.find_elements_by_tag_name("i")
    return len(find_i)

In [8]:
# 페이지 끝까지 스크롤 다운하기
def scroll_down():
    last_height = driver.execute_script("return document.body.scrollHeight") # 스크롤 높이 가져오기

    # 끝까지 스크롤 다운 한번만 진행
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [9]:
# 해당 작품으로 이동하기
def move_webtoon(num):
    driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/a').click()
    time.sleep(3)

In [10]:
# 웹툰의 글/그림/원작 정보 추출
def writer_info():
    # 글/그림 # //*[@id="content"]/div[1]/div/div[1]
    div = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/div/div[1]')
    if len(div.find_elements_by_tag_name("span")) == 3: # 글/그림이 같은 경우
        author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[1]/span[1]/a').text
        artist = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[1]/span[1]/a').text
        origin = 'X'
    elif len(div.find_elements_by_tag_name("span")) == 4: # 글/그림이 다른 경우
        author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[1]/span[1]/a').text
        artist = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[1]/span[2]/a').text
        origin = 'X'
    elif len(div.find_elements_by_tag_name("span")) == 5: # 원작까지 있는 경우 경우
        author = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[1]/span[1]/a').text
        artist = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[1]/span[2]/a').text
        origin = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[1]/span[3]/a').text
    #전체 작품 수 # //*[@id="content"]/div[1]/div/div[1]/span[{??}] # ??에는 span 수 구해서 뒤에서 2번째
    total = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/div/div[1]/span[{len(div.find_elements_by_tag_name("span"))-1}]').text

    return author, artist, origin, total

## 웹툰 information

'seq','title','descript','genre','author','artist','origin','total','star'
- 'seq': 임의의 번호
- 'title': 웹툰 제목
- 'descript': 웹툰 소개 글
- 'genre': 태그된 장르
- 'author': 글
- 'artist': 그림
- 'origin': 원작작가 (없는 경우 X로 입력)
- 'total': 전체 웹툰 수 (예고편, 후기 등 모두 포함한 수)
- 'star': 웹툰의 전체 평균 평점

## 각 웹툰 에피소드 당 평점

'title','episode','score','date'
- 'title': 웹툰 제목
- 'episode': 회차 이름
- 'score': 각 회차 당 score
- 'date': 회차 업로드 된 날짜

In [11]:
#score_data = pd.DataFrame(columns=['title','episode','score','date']) # 에피소드 평점

In [11]:
# 작품 정보 모으기
# 각 에피소드 정보 모으기
def main_info_score(num):
    global webtoon_info, score_data
    
    finish_popular() # 완결 웹툰을 인기순으로 정렬하기
    
    if num > 40: # 한페이지에 표현할 수 있는 횟수가 넘으면 스크롤 다운해주기
        repeat = num//40 # 몇번 스크롤 다운할 것인지 횟수 지정
        for i in range(repeat):
            scroll_down() # 스크롤 다운
            
    free_y_n = free(num) # 무료작품인지 확인
    if free_y_n == 0: # 무료 작품인 경우
        title = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/div/a[1]').text
        # 평균 평점
        star = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/div/div/span/span').text
        time.sleep(1)
        
        move_webtoon(num) # 해당 웹툰으로 이동하기
        
        # webtoon information
        # 작품 설명 # //*[@id="content"]/div[1]/div/div[2]/p
        descript = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/p').text
        # 장르 뽑기 # //*[@id="content"]/div[1]/div/div[2]/div/div
        genre = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/div/div').text
        # 글/그림/원작 & 전체 작품 수정보 추출
        author, artist, origin, total = writer_info()
        
        # 정보 저장하기
        webtoon_info.loc[len(webtoon_info)] = [num,title, descript, genre, author, artist, origin, total, star]
        time.sleep(2)
        ###########################################################################

        # 각 에피소드의 정보
        move = 1 # 정지하기 위한 조건
        while move==1:
            div = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[2]')
            button = div.find_elements_by_tag_name("button") # 다음 페이지로 이동할 수 있는 버튼 수 구하기

            if len(button)==12: # 이전편, 10개 목록, 다음편 으로 총 12개인경우
                move = 1 # 다음 목록으로도 이동할 수 있도록 계속 반복
                for n in range(2,12): # 2~11까지 10개 목록 반복
                    driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[2]/button[{n}]').click()
                    time.sleep(2)
                    # 각 정보 뽑기
                    ul = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul')
                    li = ul.find_elements_by_tag_name("li")
                    for num in range(1,len(li)+1):
                        episode = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul/li[{num}]/a/div[2]/p/span').text
                        score = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul/li[{num}]/a/div[2]/div/span[1]/span').text
                        date = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul/li[{num}]/a/div[2]/div/span[2]').text
                        score_data.loc[len(score_data)] = [title, episode, score, date]
                driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[2]/button[{12}]').click() # 다음목록으로 넘어가기
                time.sleep(2)
            else:
                move = 0
                for n in range(2,len(button)):
                    driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[2]/button[{n}]').click()
                    time.sleep(2)

                    # 각 정보 뽑기
                    ul = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul')
                    li = ul.find_elements_by_tag_name("li")
                    for num in range(1,len(li)+1):
                        episode = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul/li[{num}]/a/div[2]/p/span').text
                        score = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul/li[{num}]/a/div[2]/div/span[1]/span').text
                        date = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/ul/li[{num}]/a/div[2]/div/span[2]').text
                        score_data.loc[len(score_data)] = [title, episode, score, date]
    # 기다리면 무료 작품인 경우
    else:
        pass

In [12]:
for num in range(1,51):
    main_info_score(num)

In [14]:
webtoon_info

,seq,title,descript,genre,author,artist,origin,total,star
0,1,연애혁명,평범하면서 금사빠인 고등학생 순정남 공주영은 \n까칠하고 차가운 여학생 왕자림을 보...,#드라마\n#다정남\n#혐관로맨스\n#하이틴\n#학원로맨스\n#완결무료\n#완결드라마,232,232,X,442화 완결\n∙\n15세 이용가,9.86
1,2,여신강림,"네웹 대표 글로벌 인기작! 주경, 수호, 서준. 세 청춘의 두근두근 눈호강 로맨스~...",#로맨스\n#아이돌\n#드라마&영화 원작웹툰\n#인플루언서\n#삼각관계\n#완결무료,야옹이,야옹이,X,261화 완결\n∙\n12세 이용가,9.34
2,3,캐슬,"복수를 위해 칼을 갈아온 특급 킬러 '김신',\n뒷세계 '절대권력'과의 목숨 건 한...",#액션\n#복수극\n#느와르\n#완결무료\n#완결액션,정연,정연,X,121화 완결\n∙\n15세 이용가,9.97
3,4,노답소녀,재벌가 손녀인 가빈은 친구 한 명 없는 왕따에 성격까지 노답이다. \n5억원을 대가...,#로맨스\n#재벌\n#청춘로맨스\n#성장드라마\n#완결무료\n#완결로맨스,석우,석우,X,103화 완결\n∙\n전체연령가,9.64
4,5,휴재\n화산귀환,대 화산파 13대 제자.천하삼대검수 매화검존 청명. \n천하를 혼란에 빠뜨린 고금제...,#무협/사극\n#세계관\n#동양풍판타지\n#힘숨찐\n#회귀\n#이세계\n#먼치킨\n...,LICO,비가,X,73화\n∙\n15세 이용가,9.92
5,7,마녀와 용의 신혼일기,한 번의 실수로 인생을 꿰인 천재 마녀의 달콤살벌 신혼생활!\n신입과의 하룻밤으로 ...,#로맨스\n#연상연하\n#걸크러시\n#소설원작\n#완결무료\n#완결로맨스,뉴릉이,디망,바리달,127화 완결\n∙\n15세 이용가,9.96
6,8,2011 미스테리 단편,"네이버 웹툰 작가들이 들려주는 미스테리 괴담.\n올 여름, 당신의 등골이 오싹해진다.",#스릴러\n#완결무료\n#완결스릴러,네이버웹툰 작가,네이버웹툰 작가,X,30화 완결\n∙\n12세 이용가,8.45
7,9,모두에게 완자가,아무도 눈치보지 않고 사랑할 수 있는 그날이 오기를!\n\n사랑하며 살아가는 그녀들...,#일상\n#완결무료\n#완결일상,완자,완자,X,271화 완결\n∙\n12세 이용가,9.71
8,10,청춘 블라썸,“푸르지 않아도 괜찮아!”\n이제 막 청춘을 시작하고 있는 아이들.\n그리고 마냥 ...,#로맨스\n#드라마&영화 원작웹툰\n#학원로맨스\n#하이틴\n#캠퍼스로맨스\n#성장드라마,홍덕,NEMONE,X,130화 완결\n∙\n전체연령가,9.98
9,11,이별학,서른 살까지 모솔로 살다 겨우 첫 연애를 시작한 '고장가'는\n3년을 만나도 여자친...,#드라마\n#완결무료\n#완결드라마,맹물,맹물,X,54화 완결\n∙\n전체연령가,9.89


In [15]:
score_data

,title,episode,score,date
0,연애혁명,preseason 1. 모든 첫 만남은 대부분 우연이다,9.89,13.09.04
1,연애혁명,preseason 2. 간혹가다 호의를 호감으로 착각할 때가 있다,9.91,13.09.11
2,연애혁명,preseason 3. 용기있는 자만이 미인을 차지한다?,9.91,13.09.18
3,연애혁명,preseason 4. 재밌는 남자는 인기가 많다.,9.94,13.09.25
4,연애혁명,preseason 5. 사탕병,9.94,13.10.02
...,...,...,...,...
6398,MLB카툰,[프로야구 카툰] 한국시리즈 2차전,7.56,12.11.02
6399,MLB카툰,[프로야구 카툰] 한국시리즈 3차전,7.12,12.11.02
6400,MLB카툰,[프로야구 카툰] 한국시리즈 4차전,7.04,12.11.02
6401,MLB카툰,[프로야구 카툰] 한국시리즈 5차전,6.43,12.11.02


In [16]:
webtoon_info.to_csv('collected_data/fin_webtoon_info.csv', index=False)
score_data.to_csv('collected_data/fin_score_data.csv', index=False)

In [13]:
# 추가 정보
# webtoon_info.to_csv('collected_data/fin_webtoon_info_plus.csv', index=False)
# score_data.to_csv('collected_data/fin_score_data_plus.csv', index=False)

## 웹툰 별, 각 에피소드 댓글

각 회차 당
- best 댓글 15개
- 최신 댓글 45개

In [18]:
# comment_data

'title','episode','best_comment','comment'
- 'title': 웹툰 제목
- 'episode': 각 에피소드 명
- 'best_comment': best 댓글
- 'comment': 댓글
- 'comment_num': 총 댓글 수

In [247]:
comment_data = pd.DataFrame(columns=['title','episode','best_comment','comment','comment_num'])

베스트 댓글 수집 시, best댓글 클릭할 때 에러 발생  
.click()으로 하면 error발생 -> .send_keys(Keys.ENTER)로 대체

In [248]:
# 베스트 댓글 수집 (15개)
def best_comment():
    driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_sort_option_tab1"]').send_keys(Keys.ENTER) # best 댓글 클릭
    time.sleep(2)
    # 베스트 댓글이 15개가 아닐 수도 있으므로 수만큼 반복하도록 댓글 수 세기
    ul = driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul')
    li = ul.find_elements_by_tag_name("li")
    
    best_comment = ''
    for n in range(1,len(li)+1):
        txt = driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{n}]/div[1]/div/div[2]/span[2]').text+'\n'
        best_comment += txt
    return best_comment

In [249]:
# 전체 댓글 수집 (최신으로부터 300개 -> 더보기 20번 클릭)
def total_comment():
    driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_sort_option_tab2"]').click() # 전체 댓글 클릭
    time.sleep(2)
    # 전체 댓글 수가 적은 경우 미만으로만 수집하도록 하기
    repeat_num = 20 # 기본적으로 20번 모으도록 설정 20X15=300개의 댓글 수집
    comment_num = int(driver.find_element_by_xpath(f'//*[@id="cbox_module"]/div/div[1]/span').text.replace(',',''))
    if comment_num < repeat_num*15: # 모으려는 댓글 수보다 실제 댓글 수가 적을 경우
        repeat_num = comment_num//15 # 댓글 수 이내로 반복하여 수집할 수 있도록 하기
    if repeat_num == 0:
        repeat_num = 1
    
    n = 0
    total_comment = ''
    try:
        for repeat in range(repeat_num): # 15개 댓글 20번 모으기
            for fifteen in range(15): # 15번 반복
                n += 1 # 댓글 위치
                txt = driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{n}]/div[1]/div/div[2]/span[1]').text+'\n'
                total_comment += txt
            driver.find_element_by_xpath(f'//*[@id="cbox_module"]/div/div[7]/a/span').click() # 더보기 클릭
            time.sleep(2)
            scroll_down() # 스크롤 다운
            time.sleep(2)
    except:
        pass
        #print('삭제된 댓글이 있습니다.')
    return total_comment, comment_num

In [250]:
# 각 에피소드 별 댓글 모으기
def main_comment(num):
    global comment_data
    
    finish_popular() # 완결 웹툰을 인기순으로 정렬하기
    
    if num > 40: # 한페이지에 표현할 수 있는 횟수가 넘으면 스크롤 다운해주기
        repeat = num//40 # 몇번 스크롤 다운할 것인지 횟수 지정
        for i in range(repeat):
            scroll_down() # 스크롤 다운
            
    free_y_n = free(num) # 무료작품인지 확인
    if free_y_n == 0: # 무료 작품인 경우
        move_webtoon(num) # 해당 웹툰으로 이동하기
        
        # 웹툰 이름
        title = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/div/h2').text
        # 전체 회차 수 구하기
        total_num = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[1]/div[1]').text.split(' ')[1][:-1].replace(',','')
        
        # 첫 에피소드로 이동
        driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/div/a').click()
        time.sleep(2)
        scroll_down() # 제일 아래로 스크롤 다운
        
        try:
            for i in range(int(total_num)): # 에피소드 반복
                episode = driver.find_element_by_xpath(f'//*[@id="subTitle_toolbar"]').text
                # best 댓글 모으기
                best_txt = best_comment()
                # 전체 댓글 모으기
                total_txt, total_num = total_comment()

                comment_data.loc[len(comment_data)] = [title, episode, best_txt, total_txt, total_num]
                
                # 다음 에피소드로 이동
                # 음악이 삽입된 경우와 아닌 경우 나누기
                if driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').text == '다음화':
                    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').click()
                else:
                    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[5]/button').click()
            # 웹툰 목록으로 돌아가기
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)
                
        except:
            # 유료회차 창 닫기
            driver.find_element_by_xpath(f'/html/body/div[9]/div/div/div/button').send_keys(Keys.ENTER)
            # 웹툰 목록으로 돌아가기
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)

    # 기다리면 무료 작품인 경우
    else:
        pass

In [251]:
comment_data = pd.DataFrame(columns=['title','episode','best_comment','comment','comment_num'])

In [ ]:
for num in range(1,51):
    main_comment(num)

In [253]:
comment_data

,title,episode,best_comment,comment,comment_num
0,트럼프,프롤로그,이땐히아가여자인줄알았지\n이게 작가가 그렇게 에쁘다면서요.\n이해안된다고 트럼프안보...,프롤로그\n유료화 몇시부터인가요ㅠㅠ 정주행 끝까지하려면 내일까지 걸릴것같은데 흐헝\...,37151
1,트럼프,1화 멜리의 다이어리 01,아들이었다니...............반전이다 진심 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,1화 멜리의 다이어리\n정주행 ON.\n...마지막 정주행..ㅠㅠㅠ\n트럼프...진...,19106
2,트럼프,2화 멜리의 다이어리 02,고목나무할머니 진짜 고목나무처럼생겼다\n베도때는 오드아이 아니엿는데 지금은 오드아이...,2\n이제 이것만 봐도 불쌍해서 눈물 찔끔 남\n정주행하고 다시 보는데 회상 장면이...,10225
3,트럼프,3화 멜리의 다이어리 03,파란머리....개훈남이었는데...쩝...\n작가님말ㅋㅋㅋㅋㅋㅋㅋㅌㅋㅋㅋ\n충분히 피...,3\n3\n같은 나인이라그랬구나 ㅜㅜㅜㅜㅜㅜㅜㅜ\n나인이라서 히아센 앞에 나타난건가...,11223
4,트럼프,4화 멜리의 다이어리 04,............ 히아 천잰데.......?\n어린이들이 읽는 책에 여주인공이...,4\n사실 여기서부터 떡밥이았구나.. 창조주가 살려줬다는거부터가.... 테쎄라는 창...,8459
...,...,...,...,...,...
361,짝사랑 마들렌,66화,ㅋㅋㅋ 진짜 거짓말 같이 남친 등장\n은오인데요 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,마지막 짤 때뭉에 다 까먹음ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n와진짜 이렇게 내맘에 쏙드는 웹툰...,39
362,짝사랑 마들렌,67화,호준이 울엄마 사위 합격\n사랑은 정말 타이밍이구만... 호준.. 내가 데려갑니다....,호준아..\n호준이도 참 좋은 사람... 호준이의 짝도 어서 만나기를...\n호준?...,24
363,짝사랑 마들렌,68화,어머 박력있어 사장님\n아이스가 이어준 사랑~얼음 우그작 씹으러 가자\n완결각 잡히...,여긴여캐들이 개매력있네 ㅠ\n현숙씨......(찌잉)\n나 왜 울컥하냠 ..\n대표...,44
364,짝사랑 마들렌,마지막화,막화라니... 막화라니!!!!!!!!! 내 금요일 힐링이 하나 사라졌다.... 작가...,개존잼이였다\n참 따뜻한 웹툰이네요.덕분에 몽글몽글한 하루가 되었습니다. 감사합니다...,96


In [254]:
# 결과물 내보내기
# comment_data.to_csv('collected_data/fin_comment_data.csv', index=False)

### 중간에 멈출경우

In [252]:
# 멈춘 부분부터 실행
# for num in range(43,51):
#     main_comment(num)

In [243]:
# 한 웹툰만 지정해서 실행
total_num = 704 # 전체 에피소드 화수 적기
title = 'MLB카툰' # 웹툰 이름 적기

try:
    for i in range(703,int(total_num)): # 에피소드 반복 # 숫자는 현재까지 크롤링된 에피소드 이후 번호 적으면 됨
        episode = driver.find_element_by_xpath(f'//*[@id="subTitle_toolbar"]').text
        # best 댓글 모으기
        best_txt = best_comment()
        # 전체 댓글 모으기
        total_txt, total_num = total_comment()

        comment_data.loc[len(comment_data)] = [title, episode, best_txt, total_txt, total_num]

        # 다음 에피소드로 이동
        # 음악이 삽입된 경우와 아닌 경우 나누기
        if driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').text == '다음화':
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').click()
        else:
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[5]/button').click()
    # 웹툰 목록으로 돌아가기
    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)

except:
    # 유료회차 창 닫기
    driver.find_element_by_xpath(f'/html/body/div[9]/div/div/div/button').send_keys(Keys.ENTER)
    # 웹툰 목록으로 돌아가기
    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)

In [241]:
comment_data

,title,episode,best_comment,comment,comment_num
0,MLB카툰,MLB 카툰 2004시즌 <외계로의 귀환>,,2020년 12월 6일\n11111111111111111111111111111111...,109
1,MLB카툰,MLB 카툰 2004시즌 <텍사스 '영 스피어'>,,2021.04.15\n2020년 12월 6일\n저래놓고 망함\n테셰이라가 젊은 창이...,85
2,MLB카툰,MLB 카툰 2004시즌 <'사라진 거물' 엔키엘>,,2022.02.19\n2021.04.15\n2020년 12월 6일\n앤키엘은 포스트...,87
3,MLB카툰,MLB 카툰 2004시즌 <커브의 비밀>,,2021.04.15\n2020년 12월 6일\n짘ㅋㅋㅋㅋ톸ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,75
4,MLB카툰,MLB 카툰 2004시즌 <'졸린 눈' 이시이>,,만년 유망주 벨트레라니 ㅋㅋㅋㅋㅋ 지금은 명전첫턴이 확실한데 ㅋㅋㅋㅋ 인생사 어떻게...,76
...,...,...,...,...,...
691,MLB카툰,[프로야구 카툰] 플레이오프 4차전,,"삼성 썩소 ㅋㅋ\n30\n개 재미없어, 별점 반개도 엄청 많은 거야\n레알개재미없음...",30
692,MLB카툰,[프로야구 카툰] 플레이오프 정리,,ㅋㅋ\n강민호 ㅋㅋ\n막뎃\n재미없다.\n내가 부산 금정구의 윤동환이다!\n이때 까...,31
693,MLB카툰,[프로야구 카툰] 한국시리즈 1차전,,코로나는 언제 끝나려나..\n2019년 1빠\nㅇ 나\n내가 막댓이야\n응 아니야 ...,50
694,MLB카툰,[프로야구 카툰] 한국시리즈 2차전,,한화 가을야구 확정!!\n한화 최고 1등예약 한화!\n좋아요 하나 받아도 베댓되는 ...,33
